In [1]:
import pandas as pd
import sklearn.model_selection as selection
from sklearn import linear_model, tree, svm, neural_network, ensemble
import numpy as np
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

In [2]:
metric = lambda pred, true: np.sum(np.abs(pred- true)/true)
random_state = 42
use_hour = True
use_day = True
#tune_hour = True
#tune_day = True

use_linear = True
 

In [3]:

DATA_PATH = 'train_hour_day.csv'
data = pd.read_csv(DATA_PATH)
get_hour = lambda db: db['OrderedDate'].apply(lambda x: x.split(' ')[1].split(':')[0]).astype('int64')
get_day = lambda db: db['OrderedDate'].apply(lambda x: pd.to_datetime(x.split(' ')[0]).weekday()).astype('int64') 
#if use_hour:
#    data['hour']= get_hour(data)
#if use_day:
#    data['day_of_week'] = get_day(data)
    
    
TEST_PATH = 'test_hour_day.csv'
test = pd.read_csv(TEST_PATH)
#if use_hour:
#    test['hour']= get_hour(test)
#if use_day:
#    test['day_of_week'] = get_day(test)


In [4]:
#if use_hour or use_day:
#    data.to_csv('train{}.csv'.format(name_suf))
#    test.to_csv('test{}.csv'.format(name_suf))

In [5]:
cities = set(data['main_id_locality'])
get_city = lambda frame, city, maket: frame[maket['main_id_locality'] == city]

In [6]:
types = ['float64', 'int64'] if not use_linear else ['float64']
values = data.select_dtypes(include = types)
values = values.replace("NULL", np.nan)
result = values['RTA']
del values['RTA']
del values['RDA']

if not use_linear:
    del values['Id']
    if not use_hour:
        del values['hour']
    if not use_day:
        del values['day_of_week']
    del values['main_id_locality']
NOTNA = values.columns[values.notna().all()].tolist()
values = values[NOTNA]
vtest = test.select_dtypes(include = types)
vtest =  vtest.replace("NULL", np.nan)
vtest = vtest[NOTNA]

In [7]:
cvalues = {city: get_city(values, city, data) for city in cities}
cvtest = {city: get_city(vtest, city, test) for city in cities}
cresult = {city: get_city(result, city, data) for city in cities}
alllen = 0
for city in cities:
    alllen += len(data[data['main_id_locality'] == city])
assert(alllen == len(values))

In [8]:
test_size = 0.1
use_scaler = False

In [9]:
I = {}
doubled = {}
allData = {}
cvtest2 = {}
for city in cities:
    X_train, X_test, y_train, y_test = selection.train_test_split(cvalues[city], cresult[city], 
                                                                            test_size=test_size, 
                                                                            random_state=random_state)
    
    if use_scaler:
        scaler = StandardScaler()
        scaler.fit(X_train)
        eta = 0
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        cvtest[city] = scaler.transform(cvtest[city])
        allData[city] = scaler.transform(cvalues[city])
    else:
        eta = 0
        allData[city] = cvalues[city]
    poly = PolynomialFeatures(2) 
    poly.fit(X_train)
    
    n_components = 8
    pca = PCA(n_components= n_components, svd_solver='full')

    X_train2 = poly.transform(X_train)
    X_test2 = poly.transform(X_test)
    pca.fit(X_train2, y_train)
    X_train2 = pca.transform(X_train2)
    X_test2 = pca.transform(X_test2)
    cvtest2[city] = pca.transform(poly.transform(cvtest[city]))
    
    n_components = 6
    pca = PCA(n_components= n_components, svd_solver='full')
    pca.fit(X_train, y_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
    cvtest[city] = pca.transform(cvtest[city])
    tempI = {'Xtr': X_train,
            'Xte': X_test,
            'eta': eta,
            'ytr': y_train,
            'yte': y_test}
    doubled[city] = {
             'Xtr': X_train2,
            'Xte': X_test2,
            'ytr': y_train,
            'yte': y_test
    } 
    I[city] = tempI

In [10]:
Errors = {city: {'MEAN': metric(I[city]['eta'] + np.mean(I[city]['yte'] - I[city]['eta']), I[city]['yte'])} for city in cities}
res_cl = {city: {} for city in cities}

In [11]:
SGD = partial(linear_model.SGDRegressor, warm_start = True)
Forest = partial(ensemble.RandomForestRegressor, warm_start= True)
Boost = partial(ensemble.GradientBoostingRegressor, random_state = random_state, loss = 'lad', learning_rate = 0.15)
Voting = partial(ensemble.VotingRegressor)

In [12]:
def get_classifier_dict(data, cl, **kwargs):
    data['cl'] = partial(cl, **kwargs)
    return data

get_doubled = partial(get_classifier_dict, doubled)

In [13]:
#Params = {'n_estimators = 300, learning_rate = 0.15': [300, 400, 500], 'learning_rate': [0.15]}

In [14]:
#classifiers = {}
#for n in Params['n_estimators']:
#    for lr in Params['learning_rate']:
#        classifiers['Boost_{}_{}'.format(n, lr)] = partial(Boost,  n_estimators = n, learning_rate = lr)

In [15]:
#giant = partial(Voting, [('b3', Boost(max_depth = 5, random_state = 5, n_estimators = 300)),
#                                       ('b1', Boost(max_depth = 6, random_state = 1, n_estimators = 200)), 
#                                       ('b2', Boost(max_depth = 8, random_state = 2, n_estimators = 200)),
#    
#classifiers = {'vot1320': partial(Voting, [('Boost', Boost(max_depth = 8, random_state = random_state, n_estimators = 200)),
#                                       ('Lasso', Lasso()), 
#                                       ('Ridge', Ridge()),
#                                        ('SGD', SGD()),
#                                        ('Forest', Forest(max_depth = 7, random_state = random_state)),
#                                         ('Boost2', Boost(max_depth = 6, random_state = 2*random_state, n_estimators = 300))
#                                                           ])}
classifiers = {'Boost1224': partial(Boost, max_depth = 8, n_estimators = 300)}

In [ ]:
for name, classifier in classifiers.items():
    print(name)
    alllen = 0
    ytr = I[city]['ytr']
    xtr = I[city]['Xtr']
    yte = I[city]['yte']
    xte = I[city]['Xte']
    cl = classifier
    for city in cities:
        if isinstance(cl, dict):
            ytr = cl[city]['ytr']
            xtr = cl[city]['Xtr']
            yte = cl[city]['yte']
            xte = cl[city]['Xte']
            classifier = cl['cl']
        print('City ', city)
        temp_class = classifier()
        temp_class.fit(xtr, ytr)
        res_cl[city][name] = temp_class
        predictions = temp_class.predict(xte)
        err = metric(predictions, yte)
        Errors[city][name] = err
        print(err/len(predictions))
        alllen += len(predictions)
    print('----------------------')

Boost1224
City  22390


In [ ]:
ErrorsAll = None
for city in cities:
    Ercit = Errors[city]
    if ErrorsAll is None:
        ErrorsAll = Ercit
    else:
        ErrorsAll = {name: er + Ercit[name] for name, er in ErrorsAll.items()}
ErrorsAll = {name: er/alllen for name, er in ErrorsAll.items()}
for key, value in ErrorsAll.items():
    print('{}:  {}\n'.format(key, value))

In [ ]:
#for city in cities:
#    Ercit = Errors[city]
#    print(city)
#    for key, value in Ercit.items():
#        print('{}:   {}\n'.format(key, value))

In [ ]:
for best_cl in classifiers.keys():
    best_cls = {city: res_cl[city][best_cl] for city in cities}
    tempres = {}
    res = []
    for city in cities:
        tempres[city] = best_cls[city].predict(cvtest[city])
    temp_idxs = {city: 0 for city in cities}
    for idx, row in enumerate(test.itertuples()):
        tempcity = row.main_id_locality
        res.append(tempres[tempcity][temp_idxs[tempcity]])
        temp_idxs[tempcity] += 1
    fieldnames = ['ID', 'Prediction']
    res = [{'ID': i, 'Prediction': r} for i, r in enumerate(res)]
    import csv
    OUT_PATH = 'output_{}.csv'.format(best_cl)
    with open(OUT_PATH, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(res)
    out = pd.read_csv(OUT_PATH)


In [ ]:
out